In [3]:
import numpy as np
import cupy as cp
import time
import os


# 1. Definir el tamaño del vector (más de 100 millones de elementos)
SIZE = 250_000_000
print(f"Vector size: {SIZE:,} elements")


# 2. Generar el vector de ENTRADA en la CPU (NumPy)
vector_cpu = np.random.rand(SIZE).astype(np.float32) ##tipo de dato


# Verificar la GPU y CUDA
try:
   print(f"\nGPU Name: {cp.cuda.runtime.getDeviceProperties(0)['name'].decode()}")
except:
   print("\n¡ADVERTENCIA! No se detectó una GPU de NVIDIA o CUDA. ¿Activaste la GPU en el menú 'Entorno de ejecución'?")


print("---")


# ==========================================================
# A. Cómputo en CPU (Usando NumPy)
# ==========================================================
print("Ejecutando en CPU (Secuencial)...")
start_cpu = time.perf_counter()
# np.sum() es la operación secuencial en la CPU
suma_cpu = np.sum(vector_cpu)
end_cpu = time.perf_counter()
time_cpu = end_cpu - start_cpu


print(f"Suma Total (CPU): {suma_cpu}")
print(f"Tiempo CPU (ms): {time_cpu * 1000:.4f}")


# ==========================================================
# B. Cómputo en GPU (Usando CuPy / CUDA)
# ==========================================================
print("\nEjecutando en GPU (Paralelo con CUDA)...")


# **Paso 1: Transferencia de datos CPU -> GPU**
#mover vector de la ram a la vram (gpu)
start_transfer_to_gpu = time.perf_counter()
vector_gpu = cp.asarray(vector_cpu)
end_transfer_to_gpu = time.perf_counter()


# **Paso 2: Ejecución del Kernel de Reducción**
# cp.sum() invoca un Kernel CUDA de reducción optimizado (tipo árbol binario).
start_gpu_calc = time.perf_counter()
suma_gpu = cp.sum(vector_gpu)
# Sincronización: Espera a que la GPU termine todas las operaciones
cp.cuda.Stream.null.synchronize()
end_gpu_calc = time.perf_counter()
time_gpu_calc = end_gpu_calc - start_gpu_calc


# **Paso 3: Transferencia de datos GPU -> CPU**
#mover el resultado de vuelta a la CPU para imprimirlo.
suma_resultado = suma_gpu.get()
time_gpu_total = time_gpu_calc #Solo consideramos el tiempo de cómputo para el Speedup puro


print(f"Suma Total (GPU): {suma_resultado}")
print(f"Tiempo Cómputo GPU (ms): {time_gpu_calc * 1000:.4f}")
print(f"Tiempo Transferencia CPU->GPU (ms): {(end_transfer_to_gpu - start_transfer_to_gpu) * 1000:.4f}")




# ==========================================================
# C. Comparación de Rendimiento (Salida)
# ==========================================================
print("\n" + "="*50)
print("            resultados")
print("="*50)
print(f"Tiempo CPU (Secuencial): {time_cpu * 1000:.2f} ms")
print(f"Tiempo GPU (Paralelo):   {time_gpu_total * 1000:.2f} ms")


# El speedup (aceleración) es la razón de los tiempos de ejecución.
speedup = time_cpu / time_gpu_total


print(f"ACELERACIÓN (Speedup): {speedup:.2f}x")
print("==================================================")

Vector size: 250,000,000 elements

GPU Name: Tesla T4
---
Ejecutando en CPU (Secuencial)...
Suma Total (CPU): 124999696.0
Tiempo CPU (ms): 83.0545

Ejecutando en GPU (Paralelo con CUDA)...
Suma Total (GPU): 125000120.0
Tiempo Cómputo GPU (ms): 84.5776
Tiempo Transferencia CPU->GPU (ms): 189.2414

            resultados
Tiempo CPU (Secuencial): 83.05 ms
Tiempo GPU (Paralelo):   84.58 ms
ACELERACIÓN (Speedup): 0.98x
